# Trying to Condense Code

This code opens a webpage and attempts to get every link from within it

Problems with the below code: HTTPError 403, useful information may be found on the pages of links (this can be implemented fairly easily but could increase run time dramatically), still need to handle pdf's 

Thao said that there was a work around for 403's??

In [2]:
from bs4 import BeautifulSoup
import urllib
import re
 
originalURL = 'https://www.lys.org/services/lighthouse-community-school/'
html_page = urllib.request.urlopen(originalURL)

soup = BeautifulSoup(html_page, "lxml")

#Need to adjust parent URL to ensure that it can be used to filter the links in the page
def getUsefulParentURL(parentURL):
    #list of possible correct endings for the parentURL
    possibleParentEnds = ['.org', '.edu', '.com']
    
    for end in possibleParentEnds:
        #attempts to split parentURL by one of the endings
        parentURLSplit = parentURL.split(end)
        
        #if the split is successful then the parentURL is reassigned to the string before the split + the ending
        if len(parentURLSplit) > 1:
            parentURL = parentURLSplit[0] + end
            return parentURL
        
parentURL = getUsefulParentURL(originalURL)

#Finds all links in the page that are within the website and not malformed by ensuring the parentURL is in the URL.
def getUsefulLinks():
    links = []
    
    if soup.find_all('a') != None:
        for link in soup.find_all('a'):
            if link != None and link.get('href') != None and parentURL in link.get('href'):
                #print(link.get('href'))
                links.append(link.get('href'))
    links.append(originalURL)
    return links

#getUsefulLinks()

This method goes through tags and extracts the text from useful tags based on keywords. Filters tags based on the tags in the correct tag list to leave out non-text elements.

In [52]:
def findKeyWordTextNeighborTesterWithNameCheck(soup, keyWord):
    searchTxt = ''
    tagLst = []
    correctName = ['p', 'li', 'table']
    
    #This loop goes through all the text in every tag. If the text contains a keyword, then it puts that text's tag in the
    #tagLst.
    for elem in soup(text=re.compile(keyWord)):
        tagLst.append(elem.parent)
    
    #This loop goes through each tag and rips the text from that tag. If that tag is a header, then the text from the next 
    #useful tag is ripped instead. All text is saved in searchTxt
    for tag in tagLst:
        if 'h' in tag.name:
            typeTag = type(soup.find('li'))
            current = tag.next_sibling
            while current != None and  not isinstance(current, typeTag) and current.name not in correctName:
                current = current.next_sibling
            if current != None:
                searchTxt = searchTxt + current.text
        else:
            if(tag.name in correctName):
                searchTxt = searchTxt + tag.text
            
    return searchTxt

This method applies the scraping method for each keyword in a list of keywords.

In [53]:
def keyWordTextTester(keyWords, soup):
    foundTxt = ''
    
    for keyWord in keyWords:
        foundTxt = foundTxt + findKeyWordTextNeighborTesterWithNameCheck(soup, keyWord)
    
    return foundTxt

Lists for keywords for each category

In [54]:
mission = [' vision ', 'Vision ', 'our purpose', ' cause ', ' objectives ', ' mission', ' vision: ', ' mission: ', 'Mission']

curriculum = [ 'program', 'methods', 'pedagogy', 'approach', 'model', 'curriculum', 'academics', 'degree']

philosophy = ['beliefs', 'principles', 'creed', 'Values', 'philosophy', 'moral']

history = ['story', 'background', 'founded', 'established']

target = ['gifted', 'at-risk', 'ethnic background', 'target population', 'target audience']

resources = ['ESL', 'tutoring', 'after-school programs', 'available resources', 'services', 'opportunities', 'opportunity']

orgfactor = ['statistics', 'API', 'environment', 'ratio', 'average', 'female', 'fund', 'community']

Once we have the useful links we need to go loop through each one and parse that

In [55]:
#this method takes a list of links and applies the scraping method to that list. 
#WARNING: this method may take a while since it goes through every link, give it time to run before terminating it.
def mapURLs(links, keywords):
    foundTxt = ''
    for link in links:
        new_page = urllib.request.urlopen(link)
        new_soup = BeautifulSoup(new_page, "lxml")
        foundTxt = foundTxt + keyWordTextTester(keywords, new_soup)
    return foundTxt

#print(mapURLs(links))

In [56]:
school = {'mission' : [], 'curriculum': [], 'philosophy' : [], 'history' : [], 'target' : [], 'resources' : [],
          'organizational_factors' : []}


Next method is purely for timing purposes

In [57]:
def wrapper(func, *args):
    def wrapped():
        return func(*args)
    return wrapped

Code to run the parser for every keyword list and then adding the scraped text to the corresponding array in the school object.
# Warning this code takes a LONG time, don't give up on it.

In [58]:
links = getUsefulLinks()

def addTextToSchoolObject(school):
    keywordLists = [mission, curriculum, philosophy, history, target, resources, orgfactor]
    schoolArrayNames = ['mission', 'curriculum', 'philosophy', 'history', 'target', 'resources' , 'organizational_factors']

    for x in range(0, len(keywordLists)):
        school[schoolArrayNames[x]] += [mapURLs(links, keywordLists[x])]

In [59]:
import timeit

wrapped = wrapper(addTextToSchoolObject, school)
timeit.timeit(wrapped, number=1)

54.78619405739539

yikes that's a long time...

In [60]:
print(school)

{'mission': ['WCS’s primary mission is to prepare elementary students with the skills necessary for life-long learning by providing a physically and emotionally safe learning environment that supports academic risk-taking, invites student participation and structures cooperative learning experiences.WCS’s primary mission is to prepare students with the skills necessary for life-long learning by providing a physically and emotionally safe learning environment that supports academic risk-taking, invites student participation and structures cooperative learning experiences.Join WAVE and help us deliver on our mission: Building our Community through service to our students, educators and families. Check our flyer for more information.Join WAVE and help us deliver on our mission: Building our Community through service to our students, educators and families. Check our flyer for more information.amberhustead@gmail.com\nThe President shall provide vision and direction for carrying out the mis